In [53]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

#print('key:', OPENAI_API_KEY)

In [54]:
from openai import OpenAI

client = OpenAI(api_key = OPENAI_API_KEY)

response = client.responses.create(
    model='gpt-4.1',
    input='어벤져스 시리즈의 토르의 역할에 대해 한줄로 설명해줘'
)

In [31]:
response.output_text

'토르는 아스가르드의 천둥의 신으로, 막강한 힘과 묠니르 망치를 사용해 어벤져스의 핵심 멤버로 활약한다.'

In [32]:
response = client.responses.create(
    model='gpt-4.1',
    instructions='당신은 영화평론가야',
    input="어벤져스 시리즈의 토르의 역할에 대해 한줄로 설명해줘"
)
print(response.output_text)

토르는 아스가르드의 천둥의 신으로, 강력한 힘과 묠니르 망치를 앞세워 어벤져스 팀의 핵심 전사 역할을 한다.


In [40]:
response = client.responses.create(
    model='gpt-4.1',
    input=[
        {
            'role' : 'developer',
            'content' : '게임에 대한 이야기 하는 것을 좋아해'
        },
        {
            'role' : 'user', 
            'content' : '오늘은 무슨 게임을 할까?'
        }]
)
print(response.output_text)

게임 얘기라니 너무 좋아! 😊 어떤 장르를 좋아해? 가벼운 게임, 협동/파티 게임, 깊이 있는 전략 게임, RPG, 아니면 액션 게임 등등… 원하는 스타일이나 상황을 알려주면 추천해줄 수 있어!

혹시 오늘 혼자서 할 생각이야? 친구들이나 가족이랑 같이 할 계획이야?  
플랫폼(PC, 콘솔, 모바일 등)도 말해주면 더 딱 맞는 게임들을 추천해줄 수 있어!


In [41]:
completion = client.chat.completions.create(
    model='gpt-4.1',
    messages=[
        {
            'role' : 'user',
            'content' : '어벤져스 시리즈에 대해 한 문장으로 설명해줘'
        }]
)


In [42]:
#dict(completion.choices[0])
completion.choices[0].message.content

'어벤져스 시리즈는 다양한 슈퍼히어로들이 힘을 합쳐 지구와 우주를 위협하는 강력한 악당에 맞서 싸우는 마블 시네마틱 유니버스(MCU)의 대표적인 영화 시리즈입니다.'

In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

user_input = input("질문을 입력해주세요 : ")
prompt = input("프롬프트를 입력해주세요 : ")

response = client.responses.create(
    model='gpt-4.1',
    input=[
        {
            'role' : 'developer',
            "content": prompt
    
        },
        {
            'role' : 'user',
            "content" : user_input 
            
        }]
)
print(response.output_text)

안녕하ㅔ숑! 😀 반가워요! 오늘은 어떤 도움이 필요하신가요? 궁금한 거 있으면 언제든 물어봐 주세요!


File Search

In [2]:
import requests
from io import BytesIO
from openai import OpenAI
client = OpenAI()
def create_file(client, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
    # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
    )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(
            file=file_content,
            purpose="assistants"
)
    print(result.id)
    return result.id


In [3]:
file_id = create_file(client, './howto-sockets.pdf')

file-XqL3GVqVu6rY7RLRiKSt1m


In [4]:
vector_store = client.vector_stores.create(
    name='knowledge_base'
)

print(vector_store.id)

vs_685cd1a664388191a24c4b7af5652380


In [5]:
client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)

VectorStoreFile(id='file-XqL3GVqVu6rY7RLRiKSt1m', created_at=1750913447, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_685cd1a664388191a24c4b7af5652380', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [6]:
result_list = client.vector_stores.files.list(
    vector_store_id=vector_store.id
)

print(result_list)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-XqL3GVqVu6rY7RLRiKSt1m', created_at=1750913447, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_685cd1a664388191a24c4b7af5652380', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))], has_more=False, object='list', first_id='file-XqL3GVqVu6rY7RLRiKSt1m', last_id='file-XqL3GVqVu6rY7RLRiKSt1m')


In [7]:
response = client.responses.create(
    model='gpt-4.1',
    input='파이썬 코드로 소켓 만드는 방법을 간단하게 설명해줘',
    tools=[{
        "type" : "file_search",
        "vector_store_ids" : [vector_store.id]
    }],
    tool_choice='auto'
)

print(response)

Response(id='resp_685cd1a78ae8819999c5af6335e56fbe0e6a97f33172732f', created_at=1750913447.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_685cd1a816ec8199a1f91f26dee5c7cd0e6a97f33172732f', content=[ResponseOutputText(annotations=[], text="파이썬에서 소켓(socket)을 만드는 방법을 간단하게 설명하겠습니다.\n\n1. 소켓 라이브러리 임포트  \n먼저, 파이썬 내장 라이브러리인 `socket`을 불러와야 합니다.\n```python\nimport socket\n```\n\n2. 소켓 객체 생성  \n다음과 같이 소켓 객체를 생성합니다.\n```python\ns = socket.socket(socket.AF_INET, socket.SOCK_STREAM)\n```\n여기서 `AF_INET`은 IPv4를, `SOCK_STREAM`은 TCP 타입의 소켓을 의미합니다.\n\n3. 서버/클라이언트 역할에 따라 분기  \n- **서버** 용 소켓의 경우\n  - 주소, 포트에 바인딩(bind):  \n    ```python\n    s.bind(('localhost', 12345))\n    ```\n  - 연결 대기 상태로 전환(listen):  \n    ```python\n    s.listen(1)\n    ```\n  - 연결 요청을 수락(accept):  \n    ```python\n    conn, addr = s.accept()\n    ```\n\n- **클라이언트** 용 소켓의 경우  \n  - 서버에 연결(connect):  \n    ```python\n    s.co